# PROYECTO 2. RKG (RESTART KERNEL GIRLS)
---

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as mtick
import re
import sys

sys.path.append('../')
from src import soporte_variables as spv
from src import soporte_funciones as spf

pd.options.display.max_columns = None

## TRATAMIENTO Y LIMPIEZA DE LOS DATOS A ESTUDIAR

## 1. Unión de las preguntas separadas en varias columnas (q10, q39)

In [2]:
# abrimos el .csv con los datos de Data Analyst para las preguntas relevantes seleccionadas
df = pd.read_csv('../resultados/df_da_relevante.csv', index_col = 0)
df = df.reset_index()
df.sample(2)

,index,age,gender,q3,q4,q6,q7,q8,q9,q10_part_1,q10_part_2,q10_part_3,q10_part_4,q10_part_5,q10_part_6,q10_part_7,q10_part_8,q10_part_9,q10_part_10,q10_part_11,q10_part_12,q10_part_13,q10_part_14,q10_part_15,q10_part_16,q10_other,q14,q20,q21,q22,q24,q32,q34,q35,q39_part_1,q39_part_2,q39_part_3,q39_part_4,q39_part_5,q39_part_6,q39_part_7,q39_part_8,q39_part_9,q39_other,q41
1541,17065,50-54,Man,Argentina,Bachelor’s degree,10-20 years,"Python, SQL",Python,"Spyder,Jupyter Notebook",Kaggle Notebooks,Colab Notebooks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Matplotlib ,Seaborn",Energy/Mining,0-49 employees,1-2,Analyze and understand data to influence produ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Local development environments (RStudio, Jupyt..."
249,2867,30-34,Man,Nigeria,Master’s degree,< 1 years,"Python, R, SQL",R,"Visual Studio ,Jupyter Notebook, Other",Kaggle Notebooks,NaN,NaN,NaN,NaN,NaN,IBM Watson Studio,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Matplotlib ,Seaborn ,Plotly / Plotly Express ...",Computers/Technology,0-49 employees,0,None of these activities are an important par...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Local development environments (RStudio, Jupyt..."


In [3]:
# sacamos los nulos para monitorearlos a lo largo del proceso
df.isnull().sum().reset_index().T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44
index,index,age,gender,q3,q4,q6,q7,q8,q9,q10_part_1,q10_part_2,q10_part_3,q10_part_4,q10_part_5,q10_part_6,q10_part_7,q10_part_8,q10_part_9,q10_part_10,q10_part_11,q10_part_12,q10_part_13,q10_part_14,q10_part_15,q10_part_16,q10_other,q14,q20,q21,q22,q24,q32,q34,q35,q39_part_1,q39_part_2,q39_part_3,q39_part_4,q39_part_5,q39_part_6,q39_part_7,q39_part_8,q39_part_9,q39_other,q41
0,0,0,0,0,0,0,143,154,146,1508,1639,2191,2287,2124,2286,2204,2250,2267,2132,2114,2231,2281,2280,2276,2301,2206,193,114,130,150,227,1062,1069,1858,2266,2252,2278,1739,2208,1863,2104,2251,1884,2253,533


In [5]:
# creamos una lista con las respuestas de la q10 unificadas
lista_futura_q10 = []

for indice, row in df.iterrows():
    #print(indice, row.index)
    valor = ""
    for i in row.index:
        if "q10" in i:
            if type(row[i]) != float:
                row[i] = row[i].strip()
                valor += row[i] + ","
                if valor == ",":
                    valor = ''
    if valor == '':
        valor = np.nan
    try:
        lista_futura_q10.append(valor.strip(','))
    except:
        lista_futura_q10.append(valor)
lista_futura_q10[:5]

['Kaggle Notebooks,Colab Notebooks,Databricks Collaborative Notebooks',
 'Google Cloud Datalab',
 'Kaggle Notebooks,Colab Notebooks',
 'Kaggle Notebooks,Azure Notebooks',
 'IBM Watson Studio']

In [6]:
# creamos una lista con las respuestas de la q39 unificadas
lista_futura_q39 = []

for indice, row in df.iterrows():
    #print(indice, row.index)
    valor = ""
    for i in row.index:
        if "q39" in i:
            if type(row[i]) != float:
                row[i] = row[i].strip()
                valor += row[i] + ","
                if valor == ",":
                    valor = ''
    if valor == '':
        valor = np.nan
    try:
        lista_futura_q39.append(valor.strip(','))
    except:
        lista_futura_q39.append(valor)
lista_futura_q39[:5]

[nan,
 nan,
 'I do not share my work publicly',
 nan,
 'I do not share my work publicly']

In [7]:
# creamos las columnas unificadas en el dataframe y borramos las sobrantes
df['q10'] = lista_futura_q10
df['q39'] = lista_futura_q39
df.head(3)

,index,age,gender,q3,q4,q6,q7,q8,q9,q10_part_1,q10_part_2,q10_part_3,q10_part_4,q10_part_5,q10_part_6,q10_part_7,q10_part_8,q10_part_9,q10_part_10,q10_part_11,q10_part_12,q10_part_13,q10_part_14,q10_part_15,q10_part_16,q10_other,q14,q20,q21,q22,q24,q32,q34,q35,q39_part_1,q39_part_2,q39_part_3,q39_part_4,q39_part_5,q39_part_6,q39_part_7,q39_part_8,q39_part_9,q39_other,q41,q10,q39
0,16,50-54,Man,Belgium,Bachelor’s degree,20+ years,"Python, SQL",Python,"Jupyter (JupyterLab, Jupyter Notebooks, etc) ,...",Kaggle Notebooks,Colab Notebooks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Databricks Collaborative Notebooks,NaN,NaN,NaN,NaN,NaN,"Matplotlib ,Seaborn ,Plotly / Plotly Express",Energy/Mining,"1000-9,999 employees",5-9,Analyze and understand data to influence produ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Local development environments (RStudio, Jupyt...","Kaggle Notebooks,Colab Notebooks,Databricks Co...",NaN
1,32,22-24,Nonbinary,United States of America,Some college/university study without earning ...,< 1 years,R,R,RStudio,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Google Cloud Datalab,NaN,NaN,NaN,NaN,NaN,NaN,Ggplot / ggplot2,Manufacturing/Fabrication,250-999 employees,3-4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Google Cloud Datalab,NaN
2,33,30-34,Woman,Egypt,Bachelor’s degree,3-5 years,Python,R,"Notepad++,Jupyter Notebook",Kaggle Notebooks,Colab Notebooks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,Non-profit/Service,0-49 employees,0,Analyze and understand data to influence produ...,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,I do not share my work publicly,NaN,"Basic statistical software (Microsoft Excel, G...","Kaggle Notebooks,Colab Notebooks",I do not share my work publicly


In [8]:
df.drop(columns=spv.columnas_sobrantes, inplace=True)
df.sample()

,index,age,gender,q3,q4,q6,q7,q8,q9,q14,q20,q21,q22,q24,q32,q34,q35,q41,q10,q39
583,6721,55-59,Man,United States of America,Master’s degree,5-10 years,"Python, R, SQL, MATLAB",Python,"RStudio ,Visual Studio Code (VSCode) ,PyCharm ...","Matplotlib ,Seaborn",Computers/Technology,0-49 employees,1-2,Analyze and understand data to influence produ...,"MySQL , SQLite , Microsoft SQL Server , Micros...","Tableau, Microsoft Azure Synapse",Tableau,"Basic statistical software (Microsoft Excel, G...","Kaggle Notebooks,Google Cloud Notebooks (AI Pl...","GitHub,Personal blog"


In [9]:
# reordenamos las columnas
df = df.reindex(columns = ['age', 'gender', 'q3', 'q4', 'q6', 'q7', 'q8', 'q9', 'q10', 'q14', 'q20', 'q21', 'q22', 'q24', 'q32', 'q34', 'q35', 'q39', 'q41'])

In [10]:
df.isnull().sum().reset_index().T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
index,age,gender,q3,q4,q6,q7,q8,q9,q10,q14,q20,q21,q22,q24,q32,q34,q35,q39,q41
0,0,0,0,0,0,143,154,146,809,193,114,130,150,227,1062,1069,1858,1099,533


In [11]:
df.sample(3)

,age,gender,q3,q4,q6,q7,q8,q9,q10,q14,q20,q21,q22,q24,q32,q34,q35,q39,q41
124,18-21,Woman,India,Bachelor’s degree,3-5 years,"Python, SQL, C, Java, Javascript",Python,"Jupyter (JupyterLab, Jupyter Notebooks, etc) ,...","Colab Notebooks,Binder / JupyterHub,Google Clo...","Matplotlib ,Seaborn ,Plotly / Plotly Express",Other,0-49 employees,0,None of these activities are an important par...,NaN,NaN,NaN,NaN,NaN
2261,25-29,Man,Turkey,I prefer not to answer,1-3 years,"Python, SQL",Python,"Jupyter (JupyterLab, Jupyter Notebooks, etc) ,...",Kaggle Notebooks,"Matplotlib ,Seaborn ,Plotly / Plotly Express ...",Broadcasting/Communications,250-999 employees,3-4,Analyze and understand data to influence produ...,None,None,NaN,Streamlit,"Basic statistical software (Microsoft Excel, G..."
1524,18-21,Man,India,Bachelor’s degree,1-3 years,"Python, R, SQL",Python,"RStudio ,Jupyter Notebook",NaN,Matplotlib,Online Service/Internet-based Services,"1000-9,999 employees",20+,Analyze and understand data to influence produ...,NaN,NaN,NaN,NaN,"Basic statistical software (Microsoft Excel, G..."


## 2. Tratamiento de las columnas que separaremos con .explode() más adelante

In [13]:
# limpiamos la columna 'q9'
df['q9'] = df['q9'].apply(spf.limpiar_q9)
df['q9'].reset_index().sample(10)

,index,q9
1875,1875,"Visual Studio Code (VSCode) ,Jupyter Notebook"
2177,2177,Jupyter Notebook
837,837,"Notepad++,Jupyter Notebook"
229,229,"Visual Studio Code (VSCode) , Notepad++,Jupyte..."
2070,2070,None
1011,1011,"Jupyter Notebook, Other"
297,297,"Visual Studio ,Visual Studio Code (VSCode) , N..."
595,595,"Visual Studio Code (VSCode) ,PyCharm"
593,593,"RStudio ,PyCharm , Notepad++"
487,487,PyCharm


In [14]:
# limpiamos la columna 'q24'
df['q24'] = df['q24'].apply(spf.limpiar_q24)
df['q24'].reset_index().sample(10)

,index,q24
491,491,NaN
1989,1989,None of these activities are an important par...
608,608,Build and/or run the data infrastructure that...
120,120,Build and/or run the data infrastructure that...
462,462,Analyze and understand data to influence produ...
1542,1542,Analyze and understand data to influence produ...
2124,2124,Analyze and understand data to influence produ...
401,401,Analyze and understand data to influence produ...
2014,2014,Analyze and understand data to influence produ...
895,895,Analyze and understand data to influence produ...


In [15]:
# limpiamos la columna 'q41'
df['q41'] = df['q41'].apply(spf.limpiar_q41a)
df['q41'] = df['q41'].apply(spf.limpiar_q41b)
df['q41'] = df['q41'].apply(spf.limpiar_q41c)
df['q41'] = df['q41'].apply(spf.limpiar_q41d)
df['q41'] = df['q41'].apply(spf.limpiar_q41e)
df['q24'].reset_index().sample(10)

,index,q24
1790,1790,Analyze and understand data to influence produ...
900,900,Analyze and understand data to influence produ...
419,419,Analyze and understand data to influence produ...
967,967,None of these activities are an important par...
2200,2200,Other
148,148,Analyze and understand data to influence produ...
513,513,Analyze and understand data to influence produ...
1181,1181,Analyze and understand data to influence produ...
2213,2213,Analyze and understand data to influence produ...
1618,1618,Analyze and understand data to influence produ...


In [16]:
# hacemos un split en las columnas en las que más adelante haremos un .explode()
for col in spv.columnas_unidas:
    df[col] = df[col].str.split(',')

In [17]:
#chequeamos los nulos
df.isnull().sum().reset_index().T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
index,age,gender,q3,q4,q6,q7,q8,q9,q10,q14,q20,q21,q22,q24,q32,q34,q35,q39,q41
0,0,0,0,0,0,143,154,146,809,193,114,130,150,227,1062,1069,1858,1099,533


## 3. Creación de una nueva columna para agrupar los paises por continentes

In [18]:
# creamos un diccionario que asocie cada país con su continente correspondiente
# con él creamos una nueva columna con los continentes correspondientes a cada país
df['continente'] = df['q3'].map(spv.continentes)

In [19]:
df.sample(3)

,age,gender,q3,q4,q6,q7,q8,q9,q10,q14,q20,q21,q22,q24,q32,q34,q35,q39,q41,continente
28,30-34,Woman,United States of America,Bachelor’s degree,< 1 years,"[ R, SQL]",SQL,[RStudio ],"[Kaggle Notebooks, Amazon EMR Notebooks, Googl...",[ None],[Manufacturing/Fabrication],0-49 employees,1-2,[Analyze and understand data to influence prod...,"MySQL , Microsoft SQL Server",[ Tableau CRM],NaN,[I do not share my work publicly],[Basic statistical software (Microsoft Excel-G...,América del Norte
965,40-44,Man,Japan,Bachelor’s degree,3-5 years,"[Python, SQL]",SQL,"[Visual Studio Code (VSCode) , Other]",[Kaggle Notebooks],"[ Matplotlib , Seaborn ]",[Broadcasting/Communications],"1000-9,999 employees",3-4,[Analyze and understand data to influence prod...,NaN,NaN,NaN,NaN,[Basic statistical software (Microsoft Excel-G...,Asia
1708,22-24,Man,India,Bachelor’s degree,< 1 years,"[Python, SQL]",SQL,"[ Notepad++, Jupyter Notebook]",NaN,[ None],[Online Service/Internet-based Services],"10,000 or more employees",20+,NaN,NaN,NaN,NaN,NaN,NaN,Asia


## 4. Guardado de los datos

In [20]:
df.to_csv('../resultados/df_da_relevante_limpio.csv')
df.to_pickle('../resultados/df_da_relevante_limpio.pkl')